In [ ]:
from AlgorithmImports import *

class InflationGDPAssetRotation(QCAlgorithm):
    def Initialize(self):
        self.SetStartDate(2000, 1, 1)
        self.SetCash(100000)
        self.SetWarmup(TimeSpan.FromDays(365))  # Ensure 1 year of warmup data

        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol

        # Define asset classes using ETFs
        self.equities = ["SPY", "QQQ"]  # Growth stocks
        self.defensive = ["XLP", "TLT", "GLD"]  # Bonds, gold, consumer staples
        self.energy = ["XLE"]  # Energy stocks
        self.cash_equivalent = ["SHV", "BIL"]  # Money market funds
        self.real_estate = ["VNQ"]  # REITs for diversification

        self.all_assets = self.equities + self.defensive + self.energy + self.cash_equivalent + self.real_estate

        for asset in self.all_assets:
            self.AddEquity(asset, Resolution.Daily)

        self.inflation_symbol = self.AddData(Fred, "CPIAUCSL", Resolution.Daily).Symbol
        self.gdp_symbol = self.AddData(Fred, "A191RL1Q225SBEA", Resolution.Daily).Symbol

        self.inflation_data = []
        self.gdp_data = []

        self.Debug(f"Added CPI Data: {self.inflation_symbol}, GDP Data: {self.gdp_symbol}")

        # **Fix:** Convert history object to a list before checking for emptiness
        for asset in self.all_assets:
            history = list(self.History(asset, 10, Resolution.Daily))  # Convert to list
            if not history:  # Check if list is empty
                self.Debug(f"[{self.Time}] WARNING: No historical data for {asset}.")

        # Schedule monthly rebalancing
        self.Schedule.On(self.DateRules.MonthEnd(self.spy), self.TimeRules.AfterMarketOpen(self.spy, 30), self.Rebalance)

    def OnData(self, data):
        """Ensures that new economic data is stored correctly"""

        if self.inflation_symbol in data and data[self.inflation_symbol]:
            new_value = data[self.inflation_symbol].Value
            if new_value != 0 and (len(self.inflation_data) == 0 or self.inflation_data[-1] != new_value):
                self.inflation_data.append(new_value)
                self.inflation_data = self.inflation_data[-12:]  # Keep last 12 months
                self.Debug(f"[{self.Time}] Updated Inflation Data: {new_value}")

        if self.gdp_symbol in data and data[self.gdp_symbol]:
            new_value = data[self.gdp_symbol].Value
            if new_value != 0 and (len(self.gdp_data) == 0 or self.gdp_data[-1] != new_value):
                self.gdp_data.append(new_value)
                self.gdp_data = self.gdp_data[-12:]  # Keep last 12 months
                self.Debug(f"[{self.Time}] Updated GDP Data: {new_value}")

    def Rebalance(self):
        """Rebalances portfolio based on macroeconomic trends"""

        # Debug: Show stored inflation and GDP data
        self.Debug(f"[{self.Time}] Inflation Data History: {self.inflation_data}")
        self.Debug(f"[{self.Time}] GDP Data History: {self.gdp_data}")

        # Ensure at least 1 month of economic data before rebalancing
        if len(self.inflation_data) < 1 or len(self.gdp_data) < 1:
            self.Debug(f"[{self.Time}] Skipping rebalance. Not enough data.")
            return

        # Calculate trend (handle division by zero)
        if self.inflation_data[0] == 0 or self.gdp_data[0] == 0:
            self.Debug(f"[{self.Time}] Skipping rebalance due to division by zero error.")
            return

        inflation_trend = (self.inflation_data[-1] / self.inflation_data[0]) - 1
        gdp_trend = (self.gdp_data[-1] / self.gdp_data[0]) - 1

        self.Debug(f"[{self.Time}] Inflation Trend: {inflation_trend:.2f}, GDP Trend: {gdp_trend:.2f}")

        # Determine allocation confidence levels based on trend magnitudes
        inflation_confidence = min(abs(inflation_trend * 2), 1)  # Cap at 100% confidence
        gdp_confidence = min(abs(gdp_trend * 2), 1)  # Cap at 100% confidence

        # Allocate based on economic trends with confidence weighting
        if inflation_trend > 0 and gdp_trend < 0:
            self.AllocateWeighted([(self.defensive, 0.5 + 0.5 * inflation_confidence), 
                                   (self.real_estate, 0.5 - 0.5 * inflation_confidence)])
        elif inflation_trend > 0 and gdp_trend > 0:
            self.AllocateWeighted([(self.energy, 0.5 + 0.5 * inflation_confidence), 
                                   (self.equities, 0.5 - 0.5 * inflation_confidence)])
        elif inflation_trend < 0 and gdp_trend > 0:
            self.AllocateWeighted([(self.equities, 0.5 + 0.5 * gdp_confidence), 
                                   (self.real_estate, 0.5 - 0.5 * gdp_confidence)])
        else:
            self.AllocateWeighted([(self.cash_equivalent, 1.0)])

    def AllocateWeighted(self, asset_weight_pairs):
        """Executes trades with weighted asset allocation"""
        self.Liquidate()
        valid_assets = [a for asset_group, weight in asset_weight_pairs for a in asset_group if self.Securities[a].HasData]

        if not valid_assets:
            self.Debug(f"[{self.Time}] No valid assets available for trading.")
            return

        for asset_group, weight in asset_weight_pairs:
            available_assets = [a for a in asset_group if self.Securities[a].HasData]
            if available_assets:
                allocation = weight / len(available_assets)
                for asset in available_assets:
                    self.SetHoldings(asset, allocation)
                self.Debug(f"[{self.Time}] Allocating {weight * 100}% to {available_assets}")

    def ExecuteTrades(self, assets):
        """Executes trades immediately if market is open, otherwise schedules for tomorrow"""
        self.Debug(f"[{self.Time}] Checking if market is open for execution.")
        if self.IsMarketOpen("SPY"):
            self.Debug(f"[{self.Time}] Market is open. Executing trades now.")
            self.Liquidate()
            weight = 1.0 / len(assets)
            for asset in assets:
                self.SetHoldings(asset, weight)
            self.Debug(f"[{self.Time}] Executed trades: {assets}")
        else:
            self.Debug(f"[{self.Time}] Market is closed. Scheduling trades for next open session.")
            self.Schedule.On(self.DateRules.Tomorrow, self.TimeRules.AfterMarketOpen("SPY", 30), lambda: self.ExecuteTrades(assets))
